<a href="https://colab.research.google.com/github/mohamadrezanumberone/deep-compression/blob/master/Custom_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import torch
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
import torch.nn as nn
import copy
import time
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
import torchvision.models as models
import torchsummary
from sklearn.cluster import KMeans
from skimage.io import imread
from skimage.color import rgb2gray
import numpy as np
%matplotlib inline
from scipy import ndimage
import torch
from collections import OrderedDict 

# Initialization


In [3]:
batch_size = 256
num_class = 10
num_epochs = 24
feature_extract = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Dataset

In [4]:
train_dataset = CIFAR10('./CIFAR10', train= True, transform= transforms.ToTensor(), target_transform = None, download= True)
test_dataset = CIFAR10('./CIFAR10', train= False, transform= transforms.ToTensor(), target_transform = None, download= True)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10
Files already downloaded and verified


In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for batch_number, (images, labels) in enumerate(train_loader):
    print(batch_number, images.shape)

In [7]:
one_train_batch_imgs, one_train_batch_lbls = next(iter(train_loader))
print(one_train_batch_imgs.shape,"\n", one_train_batch_lbls)

torch.Size([256, 3, 32, 32]) 
 tensor([5, 4, 5, 7, 8, 7, 5, 8, 3, 6, 6, 6, 8, 1, 1, 4, 4, 1, 0, 9, 8, 5, 2, 7,
        4, 5, 5, 7, 7, 8, 5, 1, 3, 2, 8, 7, 6, 9, 8, 0, 2, 3, 1, 5, 4, 2, 7, 6,
        2, 0, 0, 7, 5, 5, 3, 3, 5, 3, 7, 4, 7, 4, 7, 9, 7, 9, 8, 9, 7, 2, 5, 3,
        5, 2, 8, 4, 1, 6, 4, 1, 2, 1, 4, 7, 2, 7, 5, 2, 6, 9, 2, 7, 3, 1, 3, 9,
        9, 5, 7, 5, 1, 4, 1, 2, 2, 4, 1, 5, 4, 9, 7, 6, 7, 6, 2, 4, 0, 7, 4, 1,
        2, 2, 2, 0, 2, 3, 6, 2, 0, 4, 4, 2, 5, 8, 0, 2, 0, 5, 1, 8, 8, 1, 6, 2,
        4, 1, 0, 9, 0, 4, 1, 3, 6, 6, 0, 1, 9, 2, 8, 3, 0, 6, 2, 6, 9, 7, 4, 8,
        9, 0, 6, 3, 3, 5, 2, 1, 5, 9, 7, 4, 9, 4, 1, 6, 9, 6, 5, 7, 9, 1, 1, 4,
        4, 0, 3, 4, 7, 7, 9, 6, 7, 3, 0, 7, 1, 7, 8, 2, 4, 2, 2, 7, 5, 1, 7, 7,
        6, 4, 7, 6, 2, 3, 2, 3, 1, 1, 7, 0, 0, 0, 6, 2, 5, 4, 5, 6, 5, 2, 1, 0,
        3, 8, 6, 4, 9, 0, 5, 3, 3, 1, 1, 3, 3, 4, 2, 5])


In [8]:
dataloaders = {'train': train_loader ,'test': test_loader}

# Model

In [ ]:
class cluster_model(nn.Module):

    def __init__(self, num_class):

        super(cluster_model, self).__init__() 

        # Layer one
        self.conv2d_1 = nn.Sequential(nn.Conv2d(3, 32, (3, 3), stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2))
        # Layer two
        self.conv2d_2 = nn.Sequential(nn.Conv2d(32, 64, (3,3), stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2))
        # Layer three
        self.conv2d_3 = nn.Sequential(nn.Conv2d(64, 128, (3,3), stride=1, padding=2),
                        nn.ReLU(),
                        nn.MaxPool2d(3, 2))
        # Layer four
        self.conv2d_4 = nn.Sequential(nn.Conv2d(128, 256, (3,3), stride=2, padding=2),
                        nn.ReLU(),
                        nn.AdaptiveAvgPool2d((1, 1)))
        
        # Fully Connected_one(Classifier)
        self.fc_1 = nn.LazyLinear(256, 10)

    def forward(self, x):
        
        # Layer_one
        y1 = self.conv2d_1(x)

        # Layer_two
        y2 = self.conv2d_2(y1)

        # Layer_three
        y3 = self.conv2d_3(y2)

        # Layer_four
        y4 = self.conv2d_4(y3)
        y4 = y4.flatten(1)

        # Fully_connected
        y = self.fc_1(y4)
        

        return y



        



In [ ]:
model = cluster_model(num_class).to(device)
model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


cluster_model(
  (conv2d_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2d_4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): AdaptiveAvgPool2d(output_size=(1, 1))
  )
  (fc_1): LazyLinear(in_features=0, out_features=256, bias=True)
)

In [ ]:
model(torch.randn(10, 3, 32, 32).to(device)).shape

torch.Size([10, 256])

In [ ]:
a = next(iter(dataloaders['train']))
a[0].shape
# model(a[0].cuda()).shape

torch.Size([256, 3, 32, 32])

# Config

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Trainer

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
model_ft, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs)

Epoch 0/43
----------
train Loss: 2.7823 Acc: 0.1000
test Loss: 2.3266 Acc: 0.1000

Epoch 1/43
----------
train Loss: 2.1911 Acc: 0.1596
test Loss: 2.1415 Acc: 0.1701

Epoch 2/43
----------
train Loss: 2.1341 Acc: 0.1796
test Loss: 2.0276 Acc: 0.2376

Epoch 3/43
----------
train Loss: 1.8955 Acc: 0.2898
test Loss: 1.7939 Acc: 0.3321

Epoch 4/43
----------
train Loss: 1.6333 Acc: 0.3973
test Loss: 1.5777 Acc: 0.4274

Epoch 5/43
----------
train Loss: 1.4430 Acc: 0.4701
test Loss: 1.3627 Acc: 0.4973

Epoch 6/43
----------
train Loss: 1.2933 Acc: 0.5329
test Loss: 1.3747 Acc: 0.5124

Epoch 7/43
----------
train Loss: 1.1761 Acc: 0.5822
test Loss: 1.2502 Acc: 0.5489

Epoch 8/43
----------
train Loss: 1.0846 Acc: 0.6166
test Loss: 1.0824 Acc: 0.6186

Epoch 9/43
----------
train Loss: 0.9992 Acc: 0.6475
test Loss: 1.1207 Acc: 0.6066

Epoch 10/43
----------
train Loss: 0.9258 Acc: 0.6750
test Loss: 1.0290 Acc: 0.6394

Epoch 11/43
----------
train Loss: 0.8631 Acc: 0.6958
test Loss: 0.9485 Acc